# UE 4268 - QISKIT Project 2

**`vos noms et prenoms`**, `matricule` et `email` 


Department of Physics - Faculty of Science - University of Yaoundé I

`Nom du Laboratoire`

`Date`

## Classification du cancer du sein avec un circuit quantique variationnel

Le cancer du sein est une forme de cancer courante et souvent agressive qui touche des millions de personnes dans le monde. Un diagnostic précoce est essentiel au succès du traitement, et les techniques d'apprentissage automatique ont été largement utilisées pour aider à classer les tumeurs du sein comme bénignes ou malignes.

Ce projet utilise un circuit quantique variationnel pour diagnostiquer, grâce au modèle d'apprentissage automatique quantique (QML, Quantum Machine Learning), le cancer du sein à l'aide de la base de données sur le [cancer du sein du Wisconsin](https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-wisconsin-diagnostic-dataset), encore appelée *Wisconsin Diagnostic Breast Cancer* (WDBC).

On rappelle qu'un circuit quantique variationnel, également appelé réseau de neurones quantiques, est constitué d'un circuit quantique fixe appelé *ansatz*, paramétré par un ensemble de variables. Ces variables sont ajustées afin de minimiser une fonction de coût, qui encode le problème à résoudre, comme la **classification** dans notre exemple. Nous optimisons ces paramètres en faisant tourner les qubits du circuit lors de l'apprentissage du modèle.

Le schéma suivant donne un bref aperçu du protocole du classificateur quantique variationnel (VQC, Variational Quantum Classifier) proposé par [Havlicek *et al.*](https://arxiv.org/abs/1804.11326).


<center> 
<img src="./Graphics/machinelearningWF.jpg" width=400 />
<img src="./Graphics/VQC_Diagram.png" width=400 />
 </center>

Le classificateur quantique variationnel est un algorithme variationnel où la valeur de la moyenne mesurée est interprétée comme la sortie d'un classificateur. 

Il est vivement conseillé de lire les notions du [Quantum Machine Learning](https://learn.qiskit.org/course/machine-learning/data-encoding) de la bibliothèque Qiskit.


### Importer les modules python basique

In [ ]:
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt
import time

## Analyse exploratoire des données - Exploratory Data Analysis (EDA)¶

Tout d'abord, explorons le jeu de données WDBC que ce projet va utiliser et voyons ce qu'il contient. Pour notre commodité, cette base de données est disponible dans scikit-learn et peut être chargé facilement.

### Charger la base de données sur le cancer du sein du Wisconsin
Utiliser la fonction `sklearn.datasets.load_breast_cancer`

In [ ]:
 # Put your code here

### Structure de la base de données

Décrire brièvement la structure de la base de données (Imprimer divers éléments et commenter à la fin).

In [ ]:
# cancer.keys

 # Put your code here


In [ ]:
# Shape of cancer data 

 # Put your code here


In [ ]:
# Feature names

 # Put your code here


In [ ]:
# cancer.DESCR

 # Put your code here


### Convertir la base de données sklearn en dataframe Pandas

On ajoutera la cible ou `target`.

In [ ]:
# Put your code here


Visualiser les 10 premières lignes de la base de données

In [ ]:
# Put your code here


### Affecter des variables indépendantes et une variable dépendante (cible)

Les $X$ seront utilisés comme données d'entrée, et les $y$ seront utilisés comme cibles de prédiction pour votre modèle ML.

In [ ]:
# Put your code here



In [ ]:
# Check the shapes of X and y

# Put your code here


Etant donné que la base de données sur le cancer du sein du Wisconsin présente de nombreuses caractéristiques (plus de 30), ce qui le rend difficile à traiter et à analyser sur un calculateur quantique avec un nombre limité de qubits. Par conséquent, pour rendre la tâche plus gérable, il faut d'abord utiliser l'**analyse en composantes principales (PCA, principal component analysis)** pour réduire la dimensionnalité de l'ensemble de données à seulement quatre variables pour le simulateur.

A cet effet, il faut normaliser les caractéristiques en supprimant la moyenne et en mettant à l'échelle la variance unitaire, ce qui est nécessaire pour le PCA.

### Normaliser les caractéristiques

Utiliser la classe `sklearn.preprocessing.StandardScaler`.

In [ ]:

# Put your code here



### Reduire la dimensionnalité à 4 avec le PCA

La dimension des données correspond au nombre de qubits nécessaires pour coder les données des feature maps quantiques à utiliser.

Utiliser la classe `sklearn.decomposition.PCA`

In [ ]:

# Put your code here



In [ ]:
# pca.explained_variance_ratio

# Put your code here



In [ ]:
# pca.singular_values

# Put your code here



### Normaliser les données

Utiliser la classe `sklearn.preprocessing.MinMaxScaler`. Sans spécifier de paramètres, elle fait correspondre les données à $[0,1]$.

In [ ]:

# Put your code here


## Entraînement d'un modèle classique d'apprentissage automatique

Avant d'entraîner un modèle, nous devons diviser la base de données en deux parties: une base de données d'entraînement et une base de données de test. Nous utiliserons la première partie pour entraîner le modèle et la seconde pour vérifier la performance de nos modèles sur des données non vues.

### Diviser les données

Utiliser 
* la fonction `sklearn.model_selection.train_test_split` qui divise aléatoirement la base de données en ensembles de données de d'entrainement (on prendra `train_size=.8`) et de test;
* et la classe `qiskit.utils.algorithms_globals` pour générer une graine aléatoire (random seed) afin d'assurer la reproductibilité à travers les exécutions.

In [ ]:

# Put your code here



### Support Vector Classifier (SVC) Classique

Entraîner les données avec [Support Vector Classifier](https://scikit-learn.org/stable/modules/svm.html) classique de scikit-learn.

In [ ]:

# Put your code here



Vérifier les performances de notre modèle classique et conclure.

In [ ]:

# Put your code here



## Entraînement d'un modèle d'apprentissage automatique quantique

On peut maintenant coder et entraîner un classificateur quantique variationnel (VQC, Variational quantum classifier). Le VQC est le classificateur variationnel quantique le plus simple de la bibliothèque Qiskit Machine Learning (*à installer*,`pip install qiskit[machine-learning] -U`).

Deux éléments centraux de la classe VQC sont la carte de caractéristiques ou *feature map* et l'*ansatz*.

* Comme les données sont classiques, elles sont constituées de bits, et non de qubits. Il faut trouver un moyen de coder les données en qubits. Ce processus est crucial si l'on veut obtenir un modèle quantique efficace. On parle généralement de *mappage* ou de correspondance, ce qui est le rôle du *feature map*. Il y a différents *feature maps* disponibles (`ZFeatureMap`, `ZZFeatureMap`, `PauliFeatureMap`) dans la bibliothèque de Qiskit. Il est possible également d'en programmer une personnalisée. Nous suggérons d'utiliser `ZZFeatureMap` (*Second-order Pauli-Z evolution circuit*). Le simulateur QASM sera utilisé.

* Une fois les données chargées, on peut appliquer un circuit quantique paramétré, également appelé un *ansatz* ou forme variationnelle. Ce circuit est un analogue direct des couches des réseaux de neurones classiques. Il a un ensemble de paramètres ou de poids réglables. Les poids sont optimisés de telle sorte qu'ils minimisent une fonction objective. Cette fonction objectif caractérise la distance entre les prédictions et les données étiquetées connues. Le circuit est basé sur des opérations unitaires et dépend de paramètres externes qui seront ajustables.  

    Étant donné un état préparé $|\psi_i\rangle$, le circuit modèle $\mathtt{U}(w)$ fait correspondre $|\psi_i\rangle$ à un autre vecteur $|\psi_f\rangle=\mathtt{U}(w)|\psi_i\rangle$. $\mathtt{U}(w)$ est constitué d'une série de portes unitaires. Nous suggérons d'utiliser ici des portes de $\mathtt{R}_y$ qui vont tourner les qubits autour de l'axe Y. L'intrication entre deux qubits sera assurée par des portes CNOT. 

### Encoder les données comme qubits

Utiliser la fonction `qiskit.circuit.library.ZZFeatureMap` pour créer un circuit quantique à 4 entrées, une entrée pour chaque feature. Imprimer le circuit décomposé en ses portes constitutives pour avoir une idée de l'apparence des feature map.

In [ ]:

# Put your code here


### Construire le circuit quantique parametré

Utiliser le circuit quantique variationnel `qiskit.circuit.library.RealAmplitutes`. Imprimer le circuit décomposé obtenu.

In [ ]:

# Put your code here


### Combiner feature map et circuit quantique parametré

Imprimer le circuit obtenu.

In [ ]:

# Put your code here


### Choisir l'optimiseur classique

On a besoin d'une routine d'optimisation classique qui modifie les valeurs de notre circuit variationnel et répète à nouveau tout le processus. C'est la boucle classique qui entraîne nos paramètres jusqu'à ce que la valeur de la fonction de coût diminue. Dans la bibliothèque Qiskit, on a les méthodes d'optimisation classiques suivantes:

* `COBYLA` - Optimisation contrainte par approximation linéaire.
* `SPSA` - Optimiseur d'approximation stochastique de perturbation simultanée (SPSA).
* `SLSQP` - Optimiseur de programmation des moindres carrés séquentiels

Il est à noter que la principale caractéristique de SPSA est l'approximation du gradient stochastique, qui ne nécessite que deux mesures de la fonction objectif, quelle que soit la dimension du problème d'optimisation. Selon la documentation Qiskit, le SPSA peut être utilisé en présence de bruit, et il est donc indiqué dans les situations impliquant une incertitude de mesure sur un calcul quantique lors de la recherche d'un minimum. C'est un fait important puisque nous sommes encore dans l'ère des NISQ.

Utiliser la méthode d'optimisation SPSA.

In [ ]:
# Put your code here



### Définir des valeurs aléatoires initiales pour les portes ("pondérations")

Utiliser la fonction `numpy.random.random` pour définir 16 valeurs aléatoires initiales pour les poids entraînables du classificateur.

In [ ]:

# Put your code here


### Encoder les étiquettes

Dans le VQC, les étiquettes peuvent être transmises sous différents formats. Il peut s'agir d'étiquettes simples, d'un tableau numpy unidimensionnel contenant des étiquettes entières comme 
> [0, 1, 2, ...]

ou d'un tableau numpy contenant des étiquettes catégorielles de type chaîne. Les libellés codés à une dimension sont également supportés. En interne, les étiquettes sont transformées en un codage *one-shot* et le classificateur est toujours entraîné sur des étiquettes *one-shot*. Ce sont par exemple, des données comme celles-ci
> [1, 0, 0], [0, 1, 0], [0, 0, 1]


La classification multi-label n'est pas supportée. Par exemple, des données comme ici
> [1, 1, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]

Utiliser la classe `sklearn.preprocessing.OneHotEncoder`

In [ ]:
# Put your code here



### Ajouter une fonction de rappel

Nous allons ajouter une fonction de rappel appelée `callback_graph`. `VQC` appellera cette fonction pour chaque évaluation de la fonction objective avec deux paramètres: les poids actuels et la valeur de la fonction objective à ces poids. Notre rappel ajoutera la valeur de la fonction objective à un tableau afin que nous puissions tracer l'itération par rapport à la valeur de la fonction objective. Le rappel mettra à jour le tracé à chaque itération. Noter que vous pouvez faire ce que vous voulez dans une fonction de rappel, tant qu'elle a la signature à deux paramètres que nous avons mentionnée ci-dessus.

In [ ]:
from IPython.display import clear_output

objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)


def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

### Choisir le simulateur quantique

Nous créons une instance de la primitive `Sampler` qui est l'implémentation de référence basée sur le vecteur d'état (`statevector`). Elle génère des quasi-probabilités à partir de circuits d'entrée.

Utiliser la classe `qiskit.primitives.Sampler` pour créer un échantillonneur soutenu par un calculateur quantique.

In [ ]:
# Put your code here


### Créer un objet VQC et l'exécuter sur le simulateur quantique

Construire le classificateur et à l'adapter ou *fitter*.

In [ ]:
# Put your code here



Tracer l'évolution temporelle de la fonction de coût et imprimer le temps d'entrainement.

Noter que l'optimisation prendra un certain temps à s'exécuter.

In [ ]:
# Put your code here



### Imprimer les métriques de la base de données

In [ ]:
# Put your code here



Comme nous pouvons le constater, les scores sont élevés et le modèle peut être utilisé pour prédire des étiquettes sur des données non vues.

### Choix d'un autre ansatz

Utiliser la classe `qiskit.circuit.library.EfficientSU2` pour construire l'*ansatz* et reprendre la simulation quantique.

In [ ]:
# Put your code here



### Conclusion¶

Dans ce projet, nous avons construit un modèle d'apprentissage automatique classique et deux quantiques. Imprimons un tableau global avec nos résultats. Commenter les résultats obtenus.

In [ ]:
print(f"Model                           | Test Score | Train Score")
print(f"SVC, 4 features                 | {train_score_c4:10.2f} | {test_score_c4:10.2f}")
print(f"VQC, 4 features, RealAmplitudes | {train_score_q4:10.2f} | {test_score_q4:10.2f}")
print(f"VQC, 4 features, EfficientSU2   | {train_score_q4_eff:10.2f} | {test_score_q4_eff:10.2f}")

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table

In [ ]:
# from qiskit.aqua.components.feature_maps import RawFeatureVector